In [1]:
import pandas as pd
import numpy as np
from plotnine import *
import plotnine
import psycopg2

# Import module for k-protoype cluster
from kmodes.kprototypes import KPrototypes

# Ignore warnings
import warnings
warnings.filterwarnings('ignore', category = FutureWarning)

# Format scientific notation from Pandas
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
# Établir la connexion à la base de données PostgreSQL
conn = psycopg2.connect(
    host="prod-rds-db.cbijryjiwdgw.eu-west-3.rds.amazonaws.com",
    database="prodkbdb",
    user="testdata",
    password="testData678341A",
    port = 5432
)

# Créer un curseur pour exécuter les requêtes
cur = conn.cursor()

# Exécuter une requête SELECT pour récupérer les données de la table
cur.execute("SELECT * from items")
rows = cur.fetchall()

# Transformer les résultats en un tableau Python
result_array = []

for row in rows:    
    result_array.append(row)


# Fermer le curseur et la connexion à la base de données
cur.close()
conn.close()


In [3]:
# transformer le tableau en array
result=np.array(result_array)


# transformer en dataframe et renommer les colonnes
data = pd.DataFrame(result, columns =['id', 'amount', 'description','date','itemName','parent','quantity','taxAmount','taxDescription','type','storeId','createdAt','updatedAt','taxRate'])

#supprimer les colonnes inutiles pour le clustering
data.drop(['id', 'date', 'parent', 'quantity','description', 'type','updatedAt'], axis = 1, inplace = True)

#changer le type de certaines colonnes
data['amount'] = data['amount'].astype(float)
data['taxAmount'] = data['taxAmount'].astype(float)
data['taxRate'] = data['taxRate'].astype(float)
data['createdAt'] = data['createdAt'].dt.strftime('%Y-%m-%d %H:%M:%S')



In [4]:
# retransformer en array
dataMatrix = data.to_numpy()

# Calculer le nombre de lignes à conserver (10% du total)
nombre_lignes_a_garder = int(0.10 * len(dataMatrix))

# Sélectionner aléatoirement 10% des indices de lignes
indices_lignes_gardees = np.random.choice(len(dataMatrix), size=nombre_lignes_a_garder, replace=False)

# Créer un nouveau tableau avec les lignes sélectionnées
nouveau_tableau = dataMatrix[indices_lignes_gardees]

In [6]:
# Obtenir les index des variables catégorielles
catColumnsPos = [data.columns.get_loc(col) for col in list(data.select_dtypes('object').columns)]

# implémenter le modèle avec 4 clusters
kproto = KPrototypes(n_jobs = -1, n_clusters = 4, init = 'Huang', random_state = 0)
kproto.fit(nouveau_tableau , categorical = catColumnsPos)

KPrototypes(gamma=133.97809833074174, init='Huang', n_clusters=4, n_jobs=-1,
            random_state=0)

In [9]:
# la fonction demandée

def clustering(id):
    
    item1 = result[result[:, 0] == id] 
    

    item = pd.DataFrame(item1, columns =['id', 'amount', 'description','date','itemName','parent','quantity','taxAmount','taxDescription','type','storeId','createdAt','updatedAt','taxRate'])
    item.drop(['id', 'date', 'parent', 'quantity','description', 'type','updatedAt'], axis = 1, inplace = True)
    item['amount'] = item['amount'].astype(float)
    item['taxAmount'] = item['taxAmount'].astype(float)
    item['taxRate'] = item['taxRate'].astype(float)
    item['createdAt'] = item['createdAt'].dt.strftime('%Y-%m-%d %H:%M:%S')
    itemMatrix = item.to_numpy()

    cluster = kproto.predict( np.array(itemMatrix), categorical= catColumnsPos)  
    print("Le nouveau point appartient au cluster:", cluster)
    print("Le centre du cluste",cluster,"est:",kproto.cluster_centroids_[cluster])

# test de la fonction
clustering('06c520a0-802f-4e39-a99d-6f24d32de1e9')

Le nouveau point appartient au cluster: [1]
Le centre du cluste [1] est: [['14.4063481677657' '0.265729470167405' '393.7897485418089'
  'california saumon avocat' 'TVA 5.5 %'
  'b4f0da25-d6f8-4fcc-8333-be8dda441c89' '2023-04-14 14:27:58']]


In [14]:
data.head()

,amount,itemName,taxAmount,taxDescription,storeId,createdAt,taxRate
0,0.000,gingembre offert,0.000,TVA 0 %,a3af4692-a7d6-41c4-8edf-05ff8f3c2d28,2023-05-07 20:40:52,0.000
1,0.000,jaint jacques,0.000,TVA 10 %,4768c91b-96f8-41f4-9e23-3f486de63184,2023-05-07 20:41:15,1000.000
2,8.000,50 campus,1.340,TVA 20 %,e3c96dbf-92b8-4b09-b895-00f410f51149,2023-05-07 20:41:59,2000.000
3,4.500,beurre-sucre roux,0.410,TVA 10 %,f9438a91-4cb8-4f29-8053-58363d9b4a0f,2023-05-07 20:42:19,1000.000
4,6.910,pink tonic,1.160,TVA 20 %,adef4d01-1b6a-4f8e-bda8-7ef743ecf184,2023-05-07 20:42:22,2000.000
